# Module 1, Section 3: Multi-Agent Architecture


In this section, you'll learn to build a multi-agent customer support system using:
- **Specialized sub-agents** focused on distinct domains (database vs. documents)
- **Supervisor agent** that smartly routes queries to the right expert
- **Tool wrapping**, enabling the supervisor to delegate tasks to sub-agents as tools
- **Testing with LangSmith traces** to see multi-agent coordination in action

By the end, you'll have a working system with:
- **Database Agent** for order, product, and customer queries
- **Documents Agent** for searching product documents and policies
- **Supervisor** for orchestration and delegation

<div align="center">
    <img src="../../static/supervisor_agent.png">
</div>


## Setup

Load environment variables:

In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Import Tools

We'll use the tools we created in previous sections, plus new Documents tools for document search:

<div align="center">
    <img src="../../static/db_rag_tools.png" alt="Schema Diagram">
</div>

**New:** Semantic Search Tools for Products & Policies

We've added two tools—`search_product_docs` and `search_policy_docs`—that let our agents search over product information and company policies using semantic search (vector database + embeddings). This makes it easy to answer user questions about docs, warranty, and returns with smart retrieval.

In [3]:
from tools.database import (
    get_order_status,
    get_order_items,
    get_product_info,
    get_order_item_price,
)
from tools.documents import search_product_docs, search_policy_docs

## 2. Build Documents Agent

Our first specialist: an agent focused on searching product documentation and policies.

In [4]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from config import DEFAULT_MODEL

# Initialize model
llm = init_chat_model(DEFAULT_MODEL)

# Create Documents Agent
docs_agent = create_agent(
    model=llm,
    tools=[search_product_docs, search_policy_docs],
    system_prompt="""You are the company policy and product information specialist for TechHub customer support.

Your role is to answer queries from a supervisor agent about product specifications, features, compatibility, 
policies (returns, warranties, shipping), and setup instructions given the tools you have been provided.
You do NOT interact directly with customers, you only interact with the supervisor agent.

Capabilities: Search product documentation and company policies.

Instructions:
- Always search the documentation to provide accurate, detailed information.
- If information is missing or not found, say so clearly.
- Do NOT make assumptions or provide information not explicitly present in the documentation.

Be accurate, concise, and specific in your replies.""",
    checkpointer=MemorySaver(),
)

Now, let's test the Documents Agent:

In [5]:
import uuid

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = docs_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's your return policy for opened electronics?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's your return policy for opened electronics?
================================== Ai Message ==================================

[{'text': "I'll search our return policy documentation for information about opened electronics.", 'type': 'text'}, {'id': 'toolu_01BTbC7zXdboJGopLXStCxPi', 'input': {'query': 'return policy opened electronics'}, 'name': 'search_policy_docs', 'type': 'tool_use'}]
Tool Calls:
  search_policy_docs (toolu_01BTbC7zXdboJGopLXStCxPi)
 Call ID: toolu_01BTbC7zXdboJGopLXStCxPi
  Args:
    query: return policy opened electronics
================================= Tool Message =================================
Name: search_policy_docs

[return_policy]
# Return Policy

At TechHub, we want you to be completely satisfied with your purchase. If you're not happy with your order, we accept returns within our specified return windows.

## Return Windows

**Unopened Electronics**
- 30 days from 

## 3. Build Database Agent

Our second specialist: an agent focused on querying structured data from the TechHub database (order status, order items, product info).

In [6]:
# Create Database Agent
db_agent = create_agent(
    model=llm,
    tools=[get_order_status, get_order_items, get_product_info, get_order_item_price],
    system_prompt="""You are the database specialist for TechHub customer support.

Your role is to answer queries from a supervisor agent about orders or products using the TechHub database tools you have been provided.
You do NOT interact directly with customers, you only interact with the supervisor agent.

Capabilities: Look up and report order status, order details (items, quantities), product prices, and product availability.

Instructions:
- Always retrieve answers directly from the database using the available tools.
- If information is missing or not found, say so clearly.
- Do NOT make assumptions or provide information not explicitly present in the database.

Be accurate, concise, and specific in your replies.""",
    checkpointer=MemorySaver(),
)

Let's test the Database Agent:

In [8]:
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}


result = db_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What items were in order ORD-2023-0002?"}
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

What items were in order ORD-2023-0002?
================================== Ai Message ==================================

[{'id': 'toolu_01TEuVM2zuodQRpK8eEXrKEf', 'input': {'order_id': 'ORD-2023-0002'}, 'name': 'get_order_items', 'type': 'tool_use'}]
Tool Calls:
  get_order_items (toolu_01TEuVM2zuodQRpK8eEXrKEf)
 Call ID: toolu_01TEuVM2zuodQRpK8eEXrKEf
  Args:
    order_id: ORD-2023-0002
================================= Tool Message =================================
Name: get_order_items

Items in order ORD-2023-0002:
- Product ID: TECH-AUD-016, Quantity: 1
- Product ID: TECH-LAP-002, Quantity: 1
- Product ID: TECH-LAP-001, Quantity: 1

================================== Ai Message ==================================

Order ORD-2023-0002 contained the following items:

1. **TECH-AUD-016** - Quantity: 1
2. **TECH-LAP-002** - Quantity: 1
3. **TECH-LAP-001** - Quantity: 1


## 4. Build Supervisor Agent

Now we'll create a supervisor agent that coordinates our specialists.

**Key insight:** Sub-agents become *tools* for the supervisor!

In [9]:
from langchain_core.tools import tool


# Wrap Database Agent as a tool
@tool(
    "database_specialist",
    description="Query TechHub database specialist for order status, order details, product prices, and product availability",
)
def call_database_specialist(query: str) -> str:
    """Call the database specialist subagent.

    Args:
        query: The question to ask the database specialist
    """
    result = db_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content


# Wrap Documents Agent as a tool
@tool(
    "documentation_specialist",
    description="Query TechHub documentation specialist to search for product specs, policies, warranties, and setup instructions",
)
def call_documentation_specialist(query: str) -> str:
    """Call the documentation specialist subagent.

    Args:
        query: The question to ask the documentation specialist
    """
    result = docs_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content

**Key insight:** The tool descriptions help the supervisor decide when to use each tool, so make them clear and specific. We return only the sub-agent’s final response, as the supervisor doesn’t need to see intermediate reasoning or tool calls.

Now, lets create the supervisor agent!

In [10]:
# Create Supervisor Agent
supervisor_agent = create_agent(
    model=llm,
    tools=[call_database_specialist, call_documentation_specialist],
    system_prompt="""You are a supervisor agent for TechHub customer support.

Your role is to interact with customers to understand their questions, use the sub-agent tools provided to 
gather information needed to answer their questions, and then provide helpful responses to the customer.

Capabilities:
- Interact with customers to understand their questions
- Use database_specialist to help answer questions about orders (status, details) and products (prices, availability)
- Use documentation_specialist to help answer questions about product specs, policies, warranties, and setup instructions


You can use multiple tools if needed to fully answer the question.
Always provide helpful, accurate, concise, and specific responses to customer questions.""",
    checkpointer=MemorySaver(),
)

## 5. Test Simple Routing

Let's test the supervisor with queries that need just ONE specialist:

In [11]:
print("Query 1: Order status (should route to Database Agent)")

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2025-0030?"}
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print("\n💡 Check LangSmith traces to see: Supervisor → database_specialist → DB Agent")

Query 1: Order status (should route to Database Agent)
================================ Human Message =================================

What's the status of order ORD-2025-0030?
================================== Ai Message ==================================

[{'id': 'toolu_01Ls1Sghjes36az3hJV4df4v', 'input': {'query': 'What is the status of order ORD-2025-0030?'}, 'name': 'database_specialist', 'type': 'tool_use'}]
Tool Calls:
  database_specialist (toolu_01Ls1Sghjes36az3hJV4df4v)
 Call ID: toolu_01Ls1Sghjes36az3hJV4df4v
  Args:
    query: What is the status of order ORD-2025-0030?
================================= Tool Message =================================
Name: database_specialist

Order **ORD-2025-0030** has the following status:

- **Status:** Processing
- **Order Date:** 2025-10-18

The order is currently being processed and hasn't shipped yet.
================================== Ai Message ==================================

Your order **ORD-2025-0030** is currently in **Pro

In [12]:
print("\nQuery 2: Product question (should route to Documents Agent)")

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's included in the box with the Logitech MX Keys keyboard?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print(
    "\n💡 Check LangSmith traces to see: Supervisor → documentation_specialist → Documents Agent"
)


Query 2: Product question (should route to Documents Agent)
================================ Human Message =================================

What's included in the box with the Logitech MX Keys keyboard?
================================== Ai Message ==================================

[{'id': 'toolu_01W1zVFxexEvP16US9kg3ub1', 'input': {'query': "Logitech MX Keys keyboard what's included in the box contents"}, 'name': 'documentation_specialist', 'type': 'tool_use'}]
Tool Calls:
  documentation_specialist (toolu_01W1zVFxexEvP16US9kg3ub1)
 Call ID: toolu_01W1zVFxexEvP16US9kg3ub1
  Args:
    query: Logitech MX Keys keyboard what's included in the box contents
================================= Tool Message =================================
Name: documentation_specialist

Based on the product documentation, here's what's included in the **Logitech MX Keys Wireless Keyboard** box:

1. **Logitech MX Keys Wireless Keyboard** - the keyboard itself
2. **Logitech Bolt USB receiver** - for wirele

## 6. Test Multi-Agent Coordination

Now the more interesting part - queries that require BOTH sub-agents!

**Parallel Execution Example:** Query that requires both Database AND Documents subagents

In [13]:
print("Query 3: Requires both Database AND Documents agents - parallel")

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Is the MacBook Air in stock? What type of processor does it have? And if I buy it, what's the return policy?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print("\n💡 Check LangSmith traces to see the parallel flow")

Query 3: Requires both Database AND Documents agents - parallel
================================ Human Message =================================

Is the MacBook Air in stock? What type of processor does it have? And if I buy it, what's the return policy?
================================== Ai Message ==================================

[{'text': "I'll help you find that information. Let me check the stock status, processor specs, and return policy for the MacBook Air.", 'type': 'text'}, {'id': 'toolu_01T9ffE2ishWB7WsNQiz7M8B', 'input': {'query': 'Is MacBook Air in stock and available?'}, 'name': 'database_specialist', 'type': 'tool_use'}, {'id': 'toolu_01BMZG6LkCtRKCyopPiiZM2Q', 'input': {'query': 'MacBook Air processor specifications'}, 'name': 'documentation_specialist', 'type': 'tool_use'}, {'id': 'toolu_01DnjNVFcaCQZPY8yJ4vFPPx', 'input': {'query': 'Return policy'}, 'name': 'documentation_specialist', 'type': 'tool_use'}]
Tool Calls:
  database_specialist (toolu_01T9ffE2ishWB7WsNQiz

**Sequential Execution Example:** Query that requires SEQUENTIAL agent execution - output from first agent feeds into second agent

This demonstrates true agent orchestration where the supervisor can't parallelize!

In [14]:
print("Query 4: Requires SEQUENTIAL coordination (DB → Documents)")

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I bought a monitor in my last order (ORD-2024-0063). Is the MacBook Air compatible with it?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()
print("\n💡 Check LangSmith traces to see SEQUENTIAL flow")

Query 4: Requires SEQUENTIAL coordination (DB → Documents)
================================ Human Message =================================

I bought a monitor in my last order (ORD-2024-0063). Is the MacBook Air compatible with it?
================================== Ai Message ==================================

[{'text': "I'll help you find out if your monitor is compatible with a MacBook Air. Let me first check what monitor you ordered, and then look up its compatibility.", 'type': 'text'}, {'id': 'toolu_0189Y8bqXY6N4QVuQzKQZVK9', 'input': {'query': 'order ORD-2024-0063 details'}, 'name': 'database_specialist', 'type': 'tool_use'}]
Tool Calls:
  database_specialist (toolu_0189Y8bqXY6N4QVuQzKQZVK9)
 Call ID: toolu_0189Y8bqXY6N4QVuQzKQZVK9
  Args:
    query: order ORD-2024-0063 details
================================= Tool Message =================================
Name: database_specialist

**Order ORD-2024-0063 Details:**

**Status & Dates:**
- Status: Delivered
- Order Date: 2024-0

#### 📦 Code Refactoring Note

The agents we built in this section (Database Agent, Documents Agent, and Supervisor) have been **refactored into the `agents/` directory** as reusable factory functions:

- `agents/db_agent.py` - Database Agent factory
- `agents/docs_agent.py` - Documents Agent factory
- `agents/supervisor_agent.py` - Supervisor Agent factory

**Why factory functions?**
- Fresh checkpointer for each instantiation (no state pollution)
- Clean imports and reusability across notebooks

In **Section 4**, we'll import these agents rather than redefining them:
```python
from agents import create_db_agent, create_docs_agent, create_supervisor_agent
```

This demonstrates how to architect reusable code! 🏗️

### Key Takeaways

#### What We Built

1. **Specialized Sub-Agents**
   - Database Agent: Expert at structured data queries
   - Documents Agent: Expert at document search
   - Each agent has focused tools and expertise

2. **Supervisor Pattern**
   - Sub-agents wrapped as tools (`@tool` decorator)
   - Supervisor routes queries to appropriate specialist(s)
   - Can orchestrate **parallel** or **sequential** coordination

3. **Coordination Patterns**
   - **Simple routing**: Single agent handles entire query
   - **Parallel coordination**: Multiple agents work independently on different parts
   - **Sequential coordination**: Output from one agent feeds into another (true orchestration!)

4. **Benefits of Multi-Agent Architecture**
   - **Separation of concerns** - Each agent has clear responsibility
   - **Easier debugging** - Traces show which agent handled what
   - **Maintainability** - Update one agent without affecting others
   - **Scalability** - Easy to add new specialist agents


#### Viewing Traces in LangSmith

Go to your LangSmith project to see:
- **Routing decisions** - Which agent(s) the supervisor called
- **Tool executions** - What tools each agent used
- **Message flow** - Complete conversation tree
- **Timing** - How long each step took
- **Sequential dependencies** - See when agents must run in order

This visibility makes multi-agent systems much easier to debug!

#### What's Next: Section 4

In **Section 4**, we'll use **LangGraph primitives** to build even more sophisticated workflows:
- Custom state management beyond messages
- Conditional routing based on state
- `interrupt()` for Human-in-the-Loop
- Customer verification workflows

